# 修改annotation的label和清除空的annotation

In [ ]:
import json
import os
import glob

def process_json_files_silent(folder_path):
    """
    静默处理文件夹中的所有JSON文件
    """
    json_files = glob.glob(os.path.join(folder_path, "*.json"))
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            shapes = data.get('shapes', [])
            
            # 删除空shape的文件
            if not shapes:
                os.remove(json_file)
                image_path = data.get('imagePath')
                if image_path:
                    image_full_path = os.path.join(folder_path, image_path)
                    if os.path.exists(image_full_path):
                        os.remove(image_full_path)
                continue
            
            # 修改swd标签
            modified = False
            for shape in data.get('shapes', []):
                if shape.get('label') == 'mayswd':
                    shape['label'] = 'swd'
                    modified = True
                    
            # 2) 删除所有 label == 'swd' 的 shape
            new_shapes = [s for s in shapes if s.get('label') != 'swd']
            if len(new_shapes) != len(shapes):
                modified = True
            data['shapes'] = new_shapes
            
            if modified:
                with open(json_file, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=2, ensure_ascii=False)
                    
        except Exception as e:
            print(f"错误处理文件 {json_file}: {str(e)}")



In [2]:
process_json_files_silent("/workspace/models/runs_yolov11_pose/cropped_objects_v5/mayswd")